In [243]:
import numpy as np

In [244]:
Reply = np.array([
    [1, 5, 5, 0, 3],
    [2, 0, 5, 0, 3],
    [0, 0, 0, 0, 0],
    [2, 0, 0, 4, 0],
    [0, 4, 2, 0, 0]
])

Reply

array([[1, 5, 5, 0, 3],
       [2, 0, 5, 0, 3],
       [0, 0, 0, 0, 0],
       [2, 0, 0, 4, 0],
       [0, 4, 2, 0, 0]])

In [245]:
Retweet = np.array([
    [0, 2, 4, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 3],
    [0, 2, 5, 0, 0]
])

Retweet

array([[0, 2, 4, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 3],
       [0, 2, 5, 0, 0]])

In [246]:
Like = np.array([
    [2, 5, 1, 0, 3],
    [0, 0, 0, 0, 0],
    [0, 5, 0, 3, 3],
    [1, 0, 0, 4, 1],
    [0, 0, 9, 0, 0]
])

Like

array([[2, 5, 1, 0, 3],
       [0, 0, 0, 0, 0],
       [0, 5, 0, 3, 3],
       [1, 0, 0, 4, 1],
       [0, 0, 9, 0, 0]])

In [247]:
Post = np.array([2, 5, 10, 4, 3])

Post

array([ 2,  5, 10,  4,  3])

# Engagement Statistics

In [248]:
# 1. Find the user who replies more than others.

reply_out = Reply.sum(axis=1)
max_reply = reply_out.argmax()

max_reply

0

In [249]:
# 2. Find the user who retweets more than others.

retweet_out = Retweet.sum(axis=1)
max_retweet = retweet_out.argmax()

max_retweet

4

In [250]:
# 3. Find users who replied or retweeted at least once.

reply_retweet_out = np.where((reply_out + retweet_out) > 0)[0]

list(reply_retweet_out)

[0, 1, 3, 4]

In [251]:
# 4. Display the number of likes each user has given to their own tweets.

nbr_self_likes = np.diag(Like)

nbr_self_likes

array([2, 0, 0, 4, 0])

In [252]:
# 5. Display the number of likes each user has given to tweets other than their own.

like_out = Like.sum(axis=1)
nbr_notself_likes = like_out - nbr_self_likes

nbr_notself_likes

array([ 9,  0, 11,  2,  9])

In [253]:
# 6. Find users who like their own tweets more than they like others' tweets.

like_more_self = np.where(nbr_self_likes > nbr_notself_likes)[0]

list(like_more_self)

[3]

In [254]:
# 7. Calculate their number.

len(like_more_self)

1

In [255]:
# 8. Display the number of interactions each user has made.

total_interactions = reply_out + retweet_out + like_out

total_interactions

array([31, 10, 11, 15, 22])

# Interaction Statistics

In [256]:
# 1. Display the maximum number of likes each user has received (from users other than themselves).

max_likes_in = Like.max(axis=0)

max_likes_in

array([2, 5, 9, 4, 3])

In [257]:

# 2. Find users who have received likes for all their tweets by at least one person other than themselves.

users_all_tweets_liked = np.where(max_likes_in >= Post)[0]

users_all_tweets_liked


array([0, 1, 3, 4], dtype=int64)

In [258]:
# 3. Calculate the number of incoming (having at least one interaction).
arcs_in = ((Reply + Retweet + Like) > 0).sum(axis=0)

arcs_in

array([3, 3, 3, 2, 4])

In [259]:
# 4. Calculate the number of outgoing arcs (having at least one interaction).
arcs_out = ((Reply + Retweet + Like) > 0).sum(axis=1)

arcs_out

array([4, 3, 3, 3, 2])

In [260]:
# 5. Find nodes where the number of incoming edges is equal to the number of outgoing edges.

in_eq_out = np.where(arcs_in == arcs_out)[0]

in_eq_out

array([1, 2], dtype=int64)

In [261]:
# 6. Display the likes they have given to each other.

likes_other = Like[:, in_eq_out][in_eq_out, :]

likes_other

array([[0, 0],
       [5, 0]])

# Popularity

In [271]:
# Create a diagonal mask for Replies and Likes to exclude self-interactions
I = np.eye(5, dtype=int)
diagonal_mask = 1 - I

Reply_nodiag = Reply * diagonal_mask
Like_nodiag = Like * diagonal_mask
Retweet_diag = Retweet * I

Retweet_diag

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [273]:
# Weighted Reply Score
Reply_score = np.where(
    Reply_nodiag > Like_nodiag, Reply_nodiag * (-0.5),  # Replies > Likes: multiply Replies by -0.5
    np.where(Reply_nodiag == Like_nodiag, Like_nodiag * (-0.25), 0)  # Replies == Likes: multiply Likes by -0.25, else 0
)

Reply_score

array([[-0.  , -1.25, -2.5 , -0.  , -0.75],
       [-1.  , -0.  , -2.5 , -0.  , -1.5 ],
       [-0.  ,  0.  , -0.  ,  0.  ,  0.  ],
       [-1.  , -0.  , -0.  , -0.  ,  0.  ],
       [-0.  , -2.  ,  0.  , -0.  , -0.  ]])

In [ ]:
# Weighted Retweet Score
Retweet_score = Retweet_diag * (-0.25) # Penalize self-retweets by -0.25

Retweet_score

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


array([[-0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0.]])

In [265]:
# Weighted Like Score
Post_half = 0.5 * Post  # Half of the number of tweets by each user
Like_score = np.where(
    Like_nodiag > Post_half, Like_nodiag * 1.5,  # Likes > threshold: multiply Likes by 1.5
    Like_nodiag  # Otherwise, keep the score as is
)

Like_score

array([[ 0. ,  7.5,  1. ,  0. ,  4.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  7.5,  0. ,  4.5,  4.5],
       [ 1. ,  0. ,  0. ,  0. ,  1. ],
       [ 0. ,  0. , 13.5,  0. ,  0. ]])

In [274]:
# Calculate the popularity score for each user for each interaction which is the average of the received scores.

avg_popularity_score = (Reply_score + Retweet_score + Like_score) / 3

avg_popularity_score

array([[ 0.        ,  2.08333333, -0.5       ,  0.        ,  1.25      ],
       [-0.33333333,  0.        , -0.83333333,  0.        , -0.5       ],
       [ 0.        ,  2.5       ,  0.        ,  1.5       ,  1.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.33333333],
       [ 0.        , -0.66666667,  4.5       ,  0.        ,  0.        ]])

In [ ]:
# Calculate the final score which is the average of the scores from different interactions.

popularity_score = avg_popularity_score.mean(axis=0)

popularity_score

array([-0.06666667,  0.78333333,  0.63333333,  0.3       ,  0.51666667])